Next, we want to see whether scLEMBAS can capture the heterogeneity of cell responses upon ligand exposure. 

In [1]:
import os

import numpy as np
import pandas as pd

import scanpy as sc
from sklearn.neighbors import NearestCentroid
from scipy.spatial.distance import cdist, pdist, squareform

import torch

import matplotlib.pyplot as plt

import sys
# lembas_path = '/nobackup/users/hmbaghda/Software/LEMBAS'
lembas_path = '/nobackup/users/hmbaghda/Software/avlant_LEMBASGPU'

sclembas_path = '/home/hmbaghda/Projects/scLEMBAS/scLEMBAS'
sys.path.insert(1, os.path.join(sclembas_path))
from model.bionetwork import format_network, SignalingModel
import utilities as utils

/tmp/ipykernel_3922131/4109938256.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
n_cores = 12
os.environ["OMP_NUM_THREADS"] = str(n_cores)
os.environ["MKL_NUM_THREADS"] = str(n_cores)
os.environ["OPENBLAS_NUM_THREADS"] = str(n_cores)
os.environ["VECLIB_MAXIMUM_THREADS"] = str(n_cores)
os.environ["NUMEXPR_NUM_THREADS"] = str(n_cores)

seed = 888
utils.set_seeds(seed = seed)
data_path = '/nobackup/users/hmbaghda/scLEMBAS/analysis'

device = 'cpu'#"cuda" if torch.cuda.is_available() else "cpu"

# Set Parameters

In [3]:
projection_amplitude_in = 3
projection_amplitude_out = 1.2

# learning rate parameters
max_iter = 5000
learning_rate = 2e-3

# other training parameters
batch_size = 10
noise_level = 10

# regularization and spectral radius params
param_lambda_L2 = 1e-6
moa_lambda_L1 = 0.1 # MoAFactor
ligand_lambda_L2 = 1e-5
uniform_lambda_L2 = 1e-4
uniform_max = 1/projection_amplitude_out
spectral_loss_factor = 1e-5
target_spectral_radius = 0.8
n_probes_spectral = 5
power_steps_spectral = 50
subset_n_spectral = 10

# Build the Model

Load Data:

In [4]:
# prior knowledge signaling network
net = pd.read_csv(os.path.join(lembas_path, 'data', 'macrophage-Model.tsv'), sep = '\t', index_col = False)

# ligand input and TF output
ligand_input = pd.read_csv(os.path.join(lembas_path, 'data', 'macrophage-Ligands.tsv'), sep='\t', low_memory=False, index_col=0)
tf_output = pd.read_csv(os.path.join(lembas_path, 'data', 'macrophage-TFs.tsv'), sep='\t', low_memory=False, index_col=0)

Let's see what the signaling network looks like:

In [5]:
stimulation_label = 'stimulation'
inhibition_label = 'inhibition'
weight_label = 'mode_of_action'
source_label = 'source'
target_label = 'target'

net[[source_label, target_label, stimulation_label, inhibition_label]].head()

,source,target,stimulation,inhibition
0,P49137,Q16539,0,0
1,Q16539,P49137,1,0
2,P31749,O15111,1,0
3,O15111,P19838,1,0
4,P19838,O15111,0,0


Let's format it to fit with the necessary inputs to the SignalingModel:

In [6]:
net = format_network(net, weight_label = weight_label, stimulation_label = stimulation_label, inhibition_label = inhibition_label)
net[[source_label, target_label, weight_label, stimulation_label, inhibition_label]].head()

,source,target,mode_of_action,stimulation,inhibition
0,P49137,Q16539,0.1,0,0
1,Q16539,P49137,1.0,1,0
2,P31749,O15111,1.0,1,0
3,O15111,P19838,1.0,1,0
4,P19838,O15111,0.1,0,0


Next, let's initialize the model and format the inputs/outputs for running the model:

In [7]:
# torch.manual_seed(seed)
# torch.cuda.manual_seed(seed)
training_parameters = {'targetSteps': 100, 'maxSteps': 150, 'expFactor':50, 'tolerance': 1e-5, 'leak':1e-2}
mod = SignalingModel(net = net,
                     X_in = ligand_input,
                     y_out = tf_output, 
                     projection_amplitude_in = projection_amplitude_in, projection_amplitude_out = projection_amplitude_out,
                     weight_label = weight_label, source_label = source_label, target_label = target_label,
                     bionet_params = training_parameters, dtype = torch.float32, 
                     device = device, seed = seed)

X_in = mod.df_to_tensor(mod.X_in)
y_out = mod.df_to_tensor(mod.y_out)

The ligand input, after filtering for nodes in the network, looks like this:

In [8]:
mod.X_in.head()

,Ligand_GC,Ligand_IC,Ligand_IFNb,Ligand_IFNg,Ligand_IL10,Ligand_IL13,Ligand_IL4,Ligand_LPSc,Ligand_P3C,Ligand_PGE2,Ligand_TNFa,Ligand_upLPS
CON,0,0,0,0,0,0,0,0,0,0,0,0
GC,1,0,0,0,0,0,0,0,0,0,0,0
IFNb,0,0,1,0,0,0,0,0,0,0,0,0
IFNb+TNFa+PGE2+P3C,0,0,1,0,0,0,0,0,1,1,1,0
IFNb+TNFa+PGE2+P3C+IFNg,0,0,1,1,0,0,0,0,1,1,1,0


The TF activity output, after filtering for nodes in the network, looks like this:

In [9]:
mod.y_out.head()

,O43524,O75030,P01100,P01106,P03372,P04637,P05412,P08047,P10070,P10242,...,Q07869,Q08050,Q12778,Q13127,Q13485,Q14186,Q16665,Q9H3D4,Q9NQB0,Q9UJU2
CON,0.543487,0.506355,0.288705,0.524944,0.508840,0.535930,0.456016,0.799885,0.465701,0.407298,...,0.479674,0.404891,0.409449,0.568049,0.439274,0.702048,0.450063,0.147630,0.384904,0.677774
GC,0.677927,0.617778,0.119494,0.724938,0.613868,0.796771,0.011826,0.439144,0.421594,0.677585,...,0.299406,0.103314,0.656781,0.440104,0.544396,0.482236,0.094289,0.250332,0.209280,0.637135
IFNb,0.384689,0.621748,0.109474,0.352545,0.558823,0.512618,0.187316,0.326541,0.430194,0.573846,...,0.505945,0.732870,0.513355,0.429696,0.509111,0.545478,0.131642,0.557754,0.465558,0.649616
IFNb+TNFa+PGE2+P3C,0.894344,0.618304,0.815526,0.029120,0.789124,0.627074,0.393195,0.551965,0.399580,0.498236,...,0.241678,0.735164,0.709700,0.383326,0.503909,0.150867,0.736157,0.556562,0.390066,0.232772
IFNb+TNFa+PGE2+P3C+IFNg,0.757642,0.506760,0.772328,0.033346,0.645125,0.762146,0.619820,0.420949,0.406377,0.303368,...,0.088150,0.655950,0.762460,0.409982,0.199154,0.199775,0.818583,0.677592,0.346423,0.165553


The forward pass looks like this:

In [10]:
# X_in = mod.df_to_tensor(mod.X_in) # ligand inputs
# X_full = mod.input_layer(X_in) # ligand inputs in signaling network
# Y_full = mod.signaling_network(X_full) # signaling network weights
# Y_hat = mod.output_layer(Y_full) # TF outputs of signaling network

# Set up the Model for Training:

In [11]:
# model setup
mod.input_layer.weights.requires_grad = False # don't learn scaling factors for the ligand input concentrations
mod.signaling_network.prescale_weights(target_radius = target_spectral_radius) # spectral radius

mod.set_device(device)
mod = mod.to(device)

# parameters
spectral_capacity = mod.signaling_network.training_params['spectralTarget']

# inputs
X_in = mod.df_to_tensor(mod.X_in)
y_out = mod.df_to_tensor(mod.y_out)
X_in = X_in.to(device)
y_out = y_out.to(device)

# loss and optimizer
loss_fn = torch.nn.MSELoss(reduction='mean')
optimizer = torch.optim.Adam(mod.parameters(), lr=1, weight_decay=0)
reset_state = optimizer.state.copy()

# mean TF (across samples) loss
mean_loss = loss_fn(torch.mean(y_out, dim=0) * torch.ones(y_out.shape, device = y_out.device), y_out)

stats = utils.initialize_progress(max_iter)
n_samples = X_in.shape[0]

# Begin training loop:

In [12]:
# # e = 0
# # for e in range(e, max_iter):
# ################################################################
# # set learning rate
# cur_lr = utils.get_lr(e, max_iter, max_height = learning_rate, start_height=learning_rate/10, end_height=1e-6, peak = 1000)
# optimizer.param_groups[0]['lr'] = cur_lr

# cur_less = []
# cur_eig = []
# train_loader = np.array_split(np.random.permutation(n_samples), np.ceil(n_samples/batch_size).astype(int))
# ################################################################
# # # iterate through batches
# # for data_index in train_loader:
# mod.train()
# optimizer.zero_grad()

# # get batch I/O
# batch_size_iter = len(data_index)
# X_in_ = X_in[data_index, :].view(batch_size_iter, X_in.shape[1])
# y_out_ = y_out[data_index, :].view(batch_size_iter, y_out.shape[1])

# test

In [13]:
e = 0

# set learning rate
cur_lr = utils.get_lr(e, max_iter, max_height = learning_rate, start_height=learning_rate/10, end_height=1e-6, peak = 1000)
optimizer.param_groups[0]['lr'] = cur_lr

cur_loss = []
cur_eig = []

np.random.seed(seed + e)
train_loader = np.array_split(np.random.permutation(n_samples), np.ceil(n_samples/batch_size).astype(int))

# iterate through batches
data_index = train_loader[0]
mod.train()
optimizer.zero_grad()

# get batch I/O
batch_size_iter = len(data_index)
X_in_ = X_in[data_index, :].view(batch_size_iter, X_in.shape[1])
y_out_ = y_out[data_index, :].view(batch_size_iter, y_out.shape[1])

# forward pass
X_full = mod.input_layer(X_in_) # transform to full network with ligand input concentrations
X_full = X_full + (noise_level * cur_lr * torch.randn(X_full.shape, device = X_full.device)) # randomly add noise to signaling network input, makes model more robust
Y_full = mod.signaling_network(X_full) # train signaling network weights
Y_hat = mod.output_layer(Y_full)

# get prediction loss
fit_loss = loss_fn(y_out_, Y_hat)

# get regularization losses
sign_reg = mod.signaling_network.sign_regularization(lambda_L1 = moa_lambda_L1) # incorrect MoA
ligand_reg = mod.ligand_regularization(lambda_L2 = ligand_lambda_L2) # ligand biases
uniform_reg = mod.uniform_regularization(lambda_L2 = uniform_lambda_L2*cur_lr, Y_full = Y_full, 
                                         target_min = 0, target_max = uniform_max) # uniform distribution
param_reg = mod.L2_reg(param_lambda_L2) # all model weights and signaling network biases
stability_loss, spectral_radius = mod.signaling_network.get_SS_loss(Y_full = Y_full, spectral_loss_factor = spectral_loss_factor,
                                                                    subset_n = subset_n_spectral, n_probes = n_probes_spectral, 
                                                                    power_steps = power_steps_spectral)

total_loss = fit_loss + sign_reg + ligand_reg + uniform_reg + param_reg + stability_loss
total_loss.backward()
mod.add_gradient_noise(noise_level = noise_level)
optimizer.step()

cur_eig.append(spectral_radius)
cur_loss.append(fit_loss.item())